In [3]:
import numpy as np
import pandas as pd
import pandas_profiling
import seaborn as sns
import scipy.stats as scs
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import sqlite3
# import stats
from sklearn.cluster import KMeans

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100

%load_ext sql
# %config InlineBackend.figure_formats = ['svg']
from IPython.display import GeoJSON

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
df = pd.read_csv('Collisions.csv')
df.head(10)

,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,-122.263950,47.506127,2001,328656,330156,3857179,Unmatched,Block,NaN,S BANGOR ST BETWEEN 55TH AVE S AND 56TH AVE S,,NaN,2,Injury Collision,NaN,2,0,0,0,1,0,0,2020-02-08T00:00:00.000Z,2/8/2020,Mid-Block (not related to intersection),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,Y
1,-122.386116,47.561089,2002,328162,329662,3856099,Unmatched,Block,NaN,SW ALASKA ST BETWEEN 42ND AVE SW AND CALIFORNI...,,NaN,1,Property Damage Only Collision,NaN,2,0,0,0,0,0,0,2020-01-21T00:00:00.000Z,1/21/2020,Mid-Block (not related to intersection),16.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,N
2,-122.351134,47.570942,2003,327702,329202,3856190,Unmatched,Block,NaN,WEST SEATTLE BR EB BETWEEN ALASKAN WY VI NB ON...,,NaN,1,Property Damage Only Collision,NaN,1,0,0,0,0,0,0,2020-01-10T00:00:00.000Z,1/10/2020,Mid-Block (not related to intersection),26.0,MOTOR VEHICLE STRUCK OBJECT IN ROAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,N
3,-122.336648,47.613376,2004,328744,330244,EA13303,Unmatched,Block,NaN,6TH AVE BETWEEN OLIVE WAY AND STEWART ST,,NaN,1,Property Damage Only Collision,NaN,4,0,0,0,0,0,0,2020-02-06T00:00:00.000Z,2/6/2020,Mid-Block (not related to intersection),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,Y
4,-122.326715,47.661352,2005,328102,329602,EA06204,Unmatched,Block,NaN,NE 45TH ST BETWEEN 2ND AVE NE AND THACKERAY PL NE,,NaN,1,Property Damage Only Collision,NaN,3,0,0,0,0,0,0,2020-01-17T00:00:00.000Z,1/17/2020,Mid-Block (but intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,N
5,-122.345006,47.618561,2006,329276,330776,EA16176,Unmatched,Intersection,29575.0,6TH AVE AND DENNY WAY,,NaN,1,Property Damage Only Collision,NaN,2,0,0,0,0,0,0,2020-02-19T00:00:00.000Z,2/19/2020,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,N
6,-122.283384,47.536615,2007,329076,330576,EA15003,Unmatched,Block,NaN,39TH AVE S BETWEEN S OTHELLO ST AND S HOLLY PA...,,NaN,0,Unknown,NaN,0,0,0,0,0,0,0,2020-02-12T00:00:00.000Z,2/12/2020,Mid-Block (not related to intersection),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,Y
7,-122.384750,47.666467,2008,1366,1366,3645761,Unmatched,Intersection,25166.0,22ND AVE NW AND SHILSHOLE AVE NW,NEI,"Not Enough Information, or Insufficient Locati...",0,Unknown,NaN,0,0,0,0,0,0,0,2013-04-02T00:00:00.000Z,4/2/2013,At Intersection (intersection related),0.0,NOT ENOUGH INFORMATION / NOT APPLICABLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,N
8,-122.334288,47.624694,2009,25800,25800,1482420,Unmatched,Intersection,28760.0,FAIRVIEW AVE N AND MERCER ST,NaN,NaN,0,Unknown,NaN,0,0,0,0,0,0,0,2004-11-14T00:00:00.000Z,11/14/2004,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,NaN,NaN,NaN,NaN,NaN,4319033.0,NaN,NaN,NaN,0,0,N
9,-122.328338,47.609478,2010,328023,329523,EA05424,Unmatched,Block,NaN,9TH AVE BETWEEN SPRING ST AND SENECA ST,NEI,"Not Enough Information, or Insufficient Locati...",1,Property Damage Only Collision,NaN,2,0,0,0,0,0,0,2020-01-17T00:00:00.000Z,1/17/2020,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,Y


In [10]:
df.shape

(217911, 40)

In [7]:
df.dtypes

X                  float64
Y                  float64
OBJECTID             int64
INCKEY               int64
COLDETKEY            int64
REPORTNO            object
STATUS              object
ADDRTYPE            object
INTKEY             float64
LOCATION            object
EXCEPTRSNCODE       object
EXCEPTRSNDESC       object
SEVERITYCODE        object
SEVERITYDESC        object
COLLISIONTYPE       object
PERSONCOUNT          int64
PEDCOUNT             int64
PEDCYLCOUNT          int64
VEHCOUNT             int64
INJURIES             int64
SERIOUSINJURIES      int64
FATALITIES           int64
INCDATE             object
INCDTTM             object
JUNCTIONTYPE        object
SDOT_COLCODE       float64
SDOT_COLDESC        object
INATTENTIONIND      object
UNDERINFL           object
WEATHER             object
ROADCOND            object
LIGHTCOND           object
PEDROWNOTGRNT       object
SDOTCOLNUM         float64
SPEEDING            object
ST_COLCODE          object
ST_COLDESC          object
S

In [11]:
df.columns

Index(['X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'STATUS',
       'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC',
       'SEVERITYCODE', 'SEVERITYDESC', 'COLLISIONTYPE', 'PERSONCOUNT',
       'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INJURIES', 'SERIOUSINJURIES',
       'FATALITIES', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE',
       'SDOT_COLDESC', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND',
       'LIGHTCOND', 'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE',
       'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

# Take a look at each column

In [12]:
df[['REPORTNO', 'STATUS',
       'ADDRTYPE', 'LOCATION', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC',
       'SEVERITYCODE', 'SEVERITYDESC', 'COLLISIONTYPE', 
       'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLDESC', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND',
       'LIGHTCOND', 'PEDROWNOTGRNT', 'SPEEDING', 'ST_COLCODE',
       'ST_COLDESC', 'HITPARKEDCAR']].nunique()

REPORTNO          217908
STATUS                 2
ADDRTYPE               3
LOCATION           25051
EXCEPTRSNCODE          2
EXCEPTRSNDESC          1
SEVERITYCODE           5
SEVERITYDESC           5
COLLISIONTYPE         10
INCDATE             5901
INCDTTM           165392
JUNCTIONTYPE           7
SDOT_COLDESC          39
INATTENTIONIND         1
UNDERINFL              4
WEATHER               10
ROADCOND               9
LIGHTCOND              8
PEDROWNOTGRNT          1
SPEEDING               1
ST_COLCODE            63
ST_COLDESC            62
HITPARKEDCAR           2
dtype: int64

In [13]:
status = list(df['STATUS'].unique())
status

['Unmatched', 'Matched']

In [14]:
addrtype = list(df['ADDRTYPE'].unique())
addrtype

['Block', 'Intersection', 'Alley', nan]

In [15]:
location = list(df['LOCATION'].unique())
location[:10]

['S BANGOR ST BETWEEN 55TH AVE S AND 56TH AVE S',
 'SW ALASKA ST BETWEEN 42ND AVE SW AND CALIFORNIA AVE SW',
 'WEST SEATTLE BR EB BETWEEN ALASKAN WY VI NB ON RP AND DELRIDGE-W SEATTLE BR EB ON RP',
 '6TH AVE BETWEEN OLIVE WAY AND STEWART ST',
 'NE 45TH ST BETWEEN 2ND AVE NE AND THACKERAY PL NE',
 '6TH AVE AND DENNY WAY',
 '39TH AVE S BETWEEN S OTHELLO ST AND S HOLLY PARK DR',
 '22ND AVE NW AND SHILSHOLE AVE NW',
 'FAIRVIEW AVE N AND MERCER ST',
 '9TH AVE BETWEEN SPRING ST AND SENECA ST']

In [16]:
severity = list(df['SEVERITYCODE'].unique())
severity

['2', '1', '0', '2b', '3', nan]

In [17]:
collisiontype = list(df['COLLISIONTYPE'].unique())
collisiontype

[nan,
 'Rear Ended',
 'Sideswipe',
 'Parked Car',
 'Angles',
 'Other',
 'Cycles',
 'Left Turn',
 'Pedestrian',
 'Right Turn',
 'Head On']

In [18]:
junctiontype = list(df['JUNCTIONTYPE'].unique())
junctiontype

['Mid-Block (not related to intersection)',
 'Mid-Block (but intersection related)',
 'At Intersection (intersection related)',
 nan,
 'Driveway Junction',
 'At Intersection (but not related to intersection)',
 'Ramp Junction',
 'Unknown']